In [92]:
import os, subprocess, random, cv2, numpy
from PIL import Image
from io import BytesIO
from time import sleep
import matplotlib.pyplot as plt

val = {'start_loc':(800,850),'start_color':(111,57,0),
       'loss_loc':(800,270),'loss_color':(99,86,131),
       'win_loc':(800,270),'win_color':(251,253,153),
       'refill_loc':(960,420),'refill_color':(38,38,38),
       'want_loc':(1065,540),'want_color':(181,11,11),
       'urg_loc':(1100,480),'urg_color':(27,83,187)}

class stats:
    runs = 0
    lost = 0
    refills = 0

def start(emu):
    os.chdir(os.getenv('LOCALAPPDATA')+'\\adb\\platform-tools\\')
    subprocess.run("adb kill-server")
    if emu == 'nox': print(subprocess.run("adb connect 127.0.0.1:62001",stdout=subprocess.PIPE,stderr=subprocess.STDOUT).stdout.decode('utf-8'))
    else: print(subprocess.run("adb devices",stdout=subprocess.PIPE,stderr=subprocess.STDOUT).stdout.decode('utf-8'))

def close():
    subprocess.run("adb kill-server")
    
def tap(x,y):
    subprocess.run("adb shell input tap {x} {y}".format(x=x,y=y))
    
def screenshot():
    im = Image.open(BytesIO(subprocess.run("adb exec-out screencap -p",stdout=subprocess.PIPE).stdout)).convert('RGB')
    im.save("screencap.png")
    return im

def pixel_color(loc):
    return screenshot().getpixel(loc)

def random_tap():
    tap(random.randrange(300,1300),random.randrange(300,500))

def color_match(one,two):
    if len([n for n,v in enumerate(one) if v-20 < two[n] < v+20])==3: return True
    else: return False

def img_match(capture,template):
    capture = cv2.cvtColor(numpy.array(capture), cv2.COLOR_RGB2GRAY)
    sift = cv2.SIFT_create()
    kp1, des1 = sift.detectAndCompute(template,None)
    kp2, des2 = sift.detectAndCompute(capture,None)
    matches = cv2.BFMatcher().knnMatch(des1,des2,k=2)
    good = [m for m,n in matches if m.distance < 0.6*n.distance] # 0.75 originally
    dst_pt = [kp2[m.trainIdx].pt for m in good]
    #plt.imshow(cv2.drawMatchesKnn(template,kp1,capture,kp2,[[m] for m in good],None,flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)),plt.show()
    return tuple([numpy.mean(x) for x in zip(*dst_pt)])

def results(stats):
    return print('Result: Won '+str(stats.runs-stats.lost)+'/'+str(stats.runs)+' battles')

def rank_up():
    random_tap() # clear rank up screen
    sleep(1)

def start_run():
    print('Starting run '+str(stats.runs+1))
    tap(1400,830) # start
    if color_match(pixel_color(val['refill_loc']),val['refill_color']):
        tap(900,650) # buy
        sleep(1)
        tap(1400,830) # start
    if False: # TODO placeholder for full inventory
        sleep(1)
        tap(1400,830) # start

def win_run():
    stats.runs += 1
    print('Win')
    random_tap() # clear victory screen
    sleep(3)
    # Wanted -> [confirm] - Hunts only
    if color_match(pixel_color(val['want_loc']),val['want_color']):
        tap(850,715) # confirm
        sleep(2)
    tap(1450,830) # confirm
    sleep(2)
    # Urgent -> [confirm] - Adventure only
    if color_match(pixel_color(val['urg_loc']),val['urg_color']):
        tap(670,650)
        sleep(1)
    tap(1400,830) # try again
    sleep(1)

def lose_run():
    stats.runs += 1
    stats.lost += 1
    print('Loss')
    tap(1400,830) # try again

In [2]:
def hunt_loop():
    print('Farming hunts')
    while True:
        im = screenshot()
        if False: rank_up() # TODO
        if color_match(im.getpixel(val['start_loc']),val['start_color']): start_run()
        elif color_match(im.getpixel(val['loss_loc']),val['loss_color']): lose_run()
        elif color_match(im.getpixel(val['win_loc']),val['win_color']): win_run()
        if stats.runs == max_runs: return results(stats)
        sleep(3)

def adventure_loop():
    print('Farming adventure')
    while True:
        im = screenshot()
        if False: rank_up() # TODO
        if color_match(im.getpixel(val['start_loc']),val['start_color']): start_run()
        elif color_match(im.getpixel(val['loss_loc']),val['loss_color']): lose_run()
        elif color_match(im.getpixel(val['win_loc']),val['win_color']):
            win_run()
            sleep(1)
            tap(1400,830) # select supporter
        if stats.runs == max_runs: return results(stats)
        sleep(3)

In [6]:
emu = 'nox' # options are 'nox' and 'bluestacks'
start(emu)

* daemon not running; starting now at tcp:5037
* daemon started successfully
connected to 127.0.0.1:62001



In [4]:
max_runs = 100
hunt_loop()